In [1]:
from rasterio.mask import mask
import os
import rasterio
import shapefile
import sys

In [ ]:
class Clipper:

    def __init__(self, orthophoto_path, shapefile_path, output_path):
        self.ORTHOMOSAIC_PATH = orthophoto_path
        self.SHAPEFILE_PATH = shapefile_path
        self.OUTPUT_PATH = output_path
        self.make_folder()

    def make_folder(self):
        if not os.path.exists(self.OUTPUT_PATH):
            os.makedirs(self.OUTPUT_PATH)

    def start(self):

        sf = shapefile.Reader(self.SHAPEFILE_PATH)
        ortho_dataset = rasterio.open(self.ORTHOMOSAIC_PATH)

        # Shapefile's loop thought records
        for plot in sf.iterShapeRecords():
            # Gathers the plot geometry
            plot_geometry = plot.shape.__geo_interface__
            interface = plot.__geo_interface__
            print(interface)

            # Gets the plot's name from the record
            plot_name = plot.record[1]
            print(plot.record)

            # Reads the orthomosaic subset that matches the plot
            cropped_image, _ = mask(ortho_dataset, [plot_geometry], crop=True)
            print("image shape ", cropped_image.shape)

            # Checks whether the image is in RGBA and reduces it to RGB if necessary
            if cropped_image.shape[0] == 4:
                cropped_image = cropped_image[:3, :, :]

            file_path = os.path.join(self.OUTPUT_PATH, f'{plot_name}.png')

            with rasterio.open(file_path, 'w', driver='PNG', height=cropped_image.shape[1], width=cropped_image.shape[2], count=3, dtype='uint8') as dst:
                dst.write(cropped_image)

In [ ]:
if __name__ == "__main__":
    orthomosaic_path = '../data/orthomosaic/raster.tif'
    shapefile_path = '../data/shapefile/all_plots.shp'
    output_path = '../data/plots'
    
    clipper = Clipper(orthomosaic_path, shapefile_path, output_path)
    clipper.start()